In [2]:
import tensorflow as tf

In [7]:
# generating example dataset as range from 0 to 10 with shift
dataset = tf.data.Dataset.range(10)
# split dataset values into windows with specified size and drop other values
# as well as create new windows by shifting dataset by 1 until reach the end 
dataset = dataset.window(5, shift=1, drop_remainder=True)
for window in dataset:
    for val in window:
        print(val.numpy(), end=" ")
    print()

0 1 2 3 4 
1 2 3 4 5 
2 3 4 5 6 
3 4 5 6 7 
4 5 6 7 8 
5 6 7 8 9 


## .batch()

In [6]:
# .batch() splits whole dataset into multiple batches
dataset2 = tf.data.Dataset.range(10)

print(list(dataset2.batch(3).as_numpy_iterator()))

print(list(dataset2.batch(3, drop_remainder=True).as_numpy_iterator()))

[array([0, 1, 2], dtype=int64), array([3, 4, 5], dtype=int64), array([6, 7, 8], dtype=int64), array([9], dtype=int64)]
[array([0, 1, 2], dtype=int64), array([3, 4, 5], dtype=int64), array([6, 7, 8], dtype=int64)]


## .map() vs .flat_map()

In [10]:
# in case of windowed dataset - it is a set of smaller Datasets
print(type(dataset))
print(type(next(iter(dataset))))

<class 'tensorflow.python.data.ops.dataset_ops.WindowDataset'>
<class 'tensorflow.python.data.ops.dataset_ops._VariantDataset'>


In [14]:
# if we want to apply batches on every window within dataset
# we need to use lambda function with .map() or .flat_map()
# flat_map vs map difference - it flattens the result
dataset_flat = dataset.flat_map(lambda window: window.batch(5))

print(type(dataset_flat), '\n')

for i in dataset_flat:
    # so we can access tensors directly
    print(i)


<class 'tensorflow.python.data.ops.dataset_ops.FlatMapDataset'> 

tf.Tensor([0 1 2 3 4], shape=(5,), dtype=int64)
tf.Tensor([1 2 3 4 5], shape=(5,), dtype=int64)
tf.Tensor([2 3 4 5 6], shape=(5,), dtype=int64)
tf.Tensor([3 4 5 6 7], shape=(5,), dtype=int64)
tf.Tensor([4 5 6 7 8], shape=(5,), dtype=int64)
tf.Tensor([5 6 7 8 9], shape=(5,), dtype=int64)


In [13]:
dataset_nonflat = dataset.map(lambda window: window.batch(5))

print(type(dataset_nonflat),'\n')

for i in dataset_nonflat:
    # in case of .map() we still get Dataset of Datasets
    print(i)
    for val in i:
        print(val)
    print()

<class 'tensorflow.python.data.ops.dataset_ops.MapDataset'> 

<_VariantDataset shapes: (None,), types: tf.int64>
tf.Tensor([0 1 2 3 4], shape=(5,), dtype=int64)

<_VariantDataset shapes: (None,), types: tf.int64>
tf.Tensor([1 2 3 4 5], shape=(5,), dtype=int64)

<_VariantDataset shapes: (None,), types: tf.int64>
tf.Tensor([2 3 4 5 6], shape=(5,), dtype=int64)

<_VariantDataset shapes: (None,), types: tf.int64>
tf.Tensor([3 4 5 6 7], shape=(5,), dtype=int64)

<_VariantDataset shapes: (None,), types: tf.int64>
tf.Tensor([4 5 6 7 8], shape=(5,), dtype=int64)

<_VariantDataset shapes: (None,), types: tf.int64>
tf.Tensor([5 6 7 8 9], shape=(5,), dtype=int64)



## create x and y

In [35]:
# create x and y
dataset_flat = dataset.flat_map(lambda window: window.batch(5))
dataset_flat = dataset_flat.map(lambda window: (window[:-1], window[-1:]))
for x,y in dataset_flat:
    print(x.numpy(),y.numpy())

[0 1 2 3] [4]
[1 2 3 4] [5]
[2 3 4 5] [6]
[3 4 5 6] [7]
[4 5 6 7] [8]
[5 6 7 8] [9]


## .shuffle()

In [37]:

for x,y in dataset_flat.shuffle(buffer_size=10):
    print(x.numpy(),y.numpy())

[2 3 4 5] [6]
[1 2 3 4] [5]
[4 5 6 7] [8]
[0 1 2 3] [4]
[3 4 5 6] [7]
[5 6 7 8] [9]


## .batch()

In [56]:
for x,y in dataset_flat.shuffle(10).batch(2):
    print("x = ", x.numpy())
    print("y = ", y.numpy())
    print()

x =  [[2 3 4 5]
 [5 6 7 8]]
y =  [[6]
 [9]]

x =  [[0 1 2 3]
 [3 4 5 6]]
y =  [[4]
 [7]]

x =  [[1 2 3 4]
 [4 5 6 7]]
y =  [[5]
 [8]]

